# WatChMaL: MC Dropout Experiments
### Iñaki Erregue Álvarez-Buhilla

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import *
from utils.utils import *

In [8]:
base_path = './data/'

labels = np.load(base_path+'/labels.npy')
test_idxs = np.load(base_path+'/indices.npy')
split_idxs = np.load(base_path+'/IWCD_mPMT_Short_2_class_idxs_xps.npz')

# 1 - Dropout in Training 

Normal, FC, All, for different dropout rates

In [3]:


#modes = ['nd', 'dfc', 'dcnn', 'd', 'mcd']
modes = ['101', '101mc01', '101mc02', '101mcfc02']

In [4]:
data_splitting(split_idxs)

Total num. of events:  2950284
Events for testing: 10.004
Events for validation: 2.399
Events for training: 87.596


In [6]:
results = []
for mode in modes:
    results.append(basic_metrics(base_path, labels, mode))

In [7]:
pd.set_option("display.precision", 4)
df = pd.DataFrame(results)

df['charge'] = np.where(df['model'].str.contains('Q'), 1, 0)
df['time'] = np.where(df['model'].str.contains('T'), 1, 0)
df['scaled'] = np.where(df['model'].str.contains('u'), 0, 1)
df['collapsed'] = np.where(df['model'].str.contains('s'), 1, 0)

df['total_score'] = df.acc+df.f1+df.auc-df.log_loss

df.sort_values(by='total_score', inplace=True, ascending=False)
df = df[['model','charge', 'time', 'scaled', 'collapsed', 'log_loss', 'acc', 'f1', 'auc', 'opt_threshold', 'total_score']].reset_index(drop=True)

df.head(len(modes))

,model,charge,time,scaled,collapsed,log_loss,acc,f1,auc,opt_threshold,total_score
0,101,0,0,1,0,0.6104,0.6646,0.6543,0.7252,0.5090,1.4337
1,101mcfc02,0,0,1,0,0.6129,0.6643,0.6568,0.7231,0.5096,1.4314
2,101mc01,0,0,1,0,0.7069,0.5177,0.5800,0.5267,0.5259,0.9176
3,101mc02,0,0,1,0,0.7588,0.4984,0.6417,0.4936,0.4363,0.8750


# 2 - MC Dropout

With previous winner, implement different dropout rates and forward passes